In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K


img_width, img_height = 224, 224
train_data_dir = '../data/train/'
validation_data_dir = '../data/validation/'
nb_train_samples = 1350
nb_validation_samples = 150
epochs = 20
batch_size = 30


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
from tensorflow.keras.layers import Input, DepthwiseConv2D, \
    Conv2D ,BatchNormalization, ReLU, AvgPool2D, Flatten, Dense

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

def mobilenet_block(x, filters, strides):
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=filters, kernel_size=1, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

INPUT_SHAPE = 224, 224, 3

input = Input(INPUT_SHAPE)
x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

x = mobilenet_block(x, filters=64, strides=2)

x = mobilenet_block(x, filters=96, strides=2)
x = mobilenet_block(x, filters=96, strides=1)

x = mobilenet_block(x, filters=128, strides=2)
# x = mobilenet_block(x, filters=256, strides=1)

# x = mobilenet_block(x, filters=512, strides=2)
for _ in range(4):
    x = mobilenet_block(x, filters=128, strides=1)
  
# x = mobilenet_block(x, filters=1024, strides=2)
# x = mobilenet_block(x, filters=1024, strides=1)

x = AvgPool2D(pool_size=7, strides=1)(x)
x = Flatten()(x)
output = Dense(units=3, activation='softmax')(x)

from tensorflow.keras import Model

model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu (ReLU)                 (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 56, 56, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 32)        128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 56, 56, 32)        0     

In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-3, momentum=0.9),
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size)

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 1350 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
45/45 [==============================] - 12s 272ms/step - loss: 2.6485 - accuracy: 0.4252 - val_loss: 1.0993 - val_accuracy: 0.3333
Epoch 2/20
45/45 [==============================] - 12s 268ms/step - loss: 0.7423 - accuracy: 0.6874 - val_loss: 1.0970 - val_accuracy: 0.3533
Epoch 3/20
45/45 [==============================] - 12s 275ms/step - loss: 0.6034 - accuracy: 0.7711 - val_loss: 1.0990 - val_accuracy: 0.3667
Epoch 4/20
45/45 [==============================] - 12s 274ms/step - loss: 0.5106 - accuracy: 0.8104 - val_loss: 1.1388 - val_accuracy: 0.3267
Epoch 5/20
45/45 [==============================] - 13s 283ms/step - loss: 0.4567 - accuracy: 0.8348 - val_loss: 1.6222 - val_accuracy: 0.5067
Epoch 6/20
45/45 [==============================] - 13s 282ms/step - loss: 0.4275 - accuracy: 0.8444 - val_loss: 1.4350 - val_

In [4]:
model.save("./mobilenet_47.h5")